In [3]:
import pandas as pd
import sys
import os
import numpy as np
import xarray as xr
import zarr
import glob
from rechunker import rechunk


import dask
from ipywidgets import interact
from dask_jobqueue import SLURMCluster
from dask.distributed import Client, progress
from dask import delayed, compute
import dask.array as da
import warnings
from re import split
import matplotlib.pyplot as plt
import time
warnings.filterwarnings('ignore')
sys.path.insert(1, f"{os.path.abspath(os.path.join(os.path.abspath(''), '../'))}")
from src.utils import time_3d, get_pars_from_ini
location = split(', |_|-|!', os.popen('hostname').read())[0].replace("\n", "")
path_data = get_pars_from_ini(campaign='loc')[location]['path_data']
plt.rcParams['animation.html'] = 'jshtml'
%matplotlib inline


In [4]:
cluster = SLURMCluster(queue="seseml",
                       memory='20GB',
                       cores=20,
                       processes=1,
                       walltime='01:40:00',
                       scheduler_options={'host': '172.22.179.3:7222', 'dashboard_address': ':7330'})


In [5]:
cluster.scale(4)
cluster

In [7]:
client = Client(cluster)
client

Client Scheduler: tcp://172.22.179.3:7222 Dashboard: http://172.22.179.3:7330/status,Cluster Workers: 4 Cores: 80 Memory: 74.52 GiB


In [8]:
folders = [i for i in glob.glob(f'{path_data}/zarr/*/*') if i.endswith('.zarr')]
folders[-7]

'/data/keeling/a/alfonso8/gpm/camp2ex/zarr/KUsKAs_Wn/lores.zarr'

In [9]:
!du -sch /data/keeling/a/alfonso8/gpm/camp2ex/zarr/KUsKAs_Wn/lores.zarr

78G	/data/keeling/a/alfonso8/gpm/camp2ex/zarr/KUsKAs_Wn/lores.zarr
78G	total


In [10]:
source_group = zarr.open(folders[-7])

In [11]:
target_chunks = {}
for i in source_group.array_keys():
    dims = {}
    dim = source_group[i].attrs.asdict()['_ARRAY_DIMENSIONS']
    for k in dim:
        if k == 'time':
            dims[k] = 2000
        else:
            dims[k] = -1
    
    target_chunks[i] = dims

In [12]:
target_chunks

{'DR': {'bin_size': -1},
 'Xat_km': {'cross_track': -1, 'time': 2000},
 'alt3D': {'range': -1, 'cross_track': -1, 'time': 2000},
 'alt3d': {'range': -1, 'cross_track': -1, 'time': 2000},
 'alt_nav': {'cross_track': -1, 'time': 2000},
 'altsurf14': {'cross_track': -1, 'time': 2000},
 'altsurf35': {'cross_track': -1, 'time': 2000},
 'azimuth': {'cross_track': -1, 'time': 2000},
 'beamnum': {'cross_track': -1, 'time': 2000},
 'cross_track': {'cross_track': -1},
 'drift': {'cross_track': -1, 'time': 2000},
 'elevation': {'cross_track': -1, 'time': 2000},
 'gsp_mps': {'cross_track': -1, 'time': 2000},
 'ibeam_hires': {'bin_size': -1},
 'ipc14': {'cross_track': -1, 'time': 2000},
 'ipc35': {'cross_track': -1, 'time': 2000},
 'isc14': {'cross_track': -1, 'time': 2000},
 'isc35': {'cross_track': -1, 'time': 2000},
 'isurf': {'cross_track': -1, 'time': 2000},
 'isurf14': {'cross_track': -1, 'time': 2000},
 'isurf35': {'cross_track': -1, 'time': 2000},
 'lat': {'cross_track': -1, 'time': 2000},


In [13]:
max_mem = '1GB'
target_store = '/data/keeling/a/alfonso8/gpm/camp2ex/zarr_1/lores_rechunked.zarr'
temp_store = '/data/keeling/a/alfonso8/gpm/camp2ex/zarr_1/temp'

In [15]:
ds_source = xr.open_zarr(folders[-7])

In [16]:
ds_source

,Array,Chunk
Bytes,7.63 GiB,467.58 kiB
Shape,"(456, 25, 89835)","(114, 7, 75)"
Count,19169 Tasks,19168 Chunks
Type,float64,numpy.ndarray
,Array,Chunk
Bytes,7.63 GiB,467.58 kiB
Shape,"(456, 25, 89835)","(114, 7, 75)"
Count,19169 Tasks,19168 Chunks
Type,float64,numpy.ndarray
,Array,Chunk


In [18]:
array_plan = rechunk(source=ds_source, 
                     target_chunks=target_chunks, 
                     max_mem=max_mem, 
                     target_store=target_store, 
                     temp_store=temp_store)
array_plan


,Array,Chunk
Bytes,7.63 GiB,467.58 kiB
Shape,"(456, 25, 89835)","(114, 7, 75)"
Count,19169 Tasks,19168 Chunks
Type,float64,numpy.ndarray
,Array,Chunk
Bytes,7.63 GiB,467.58 kiB
Shape,"(456, 25, 89835)","(114, 7, 75)"
Count,19169 Tasks,19168 Chunks
Type,float64,numpy.ndarray
,Array,Chunk


In [19]:
array_plan.execute()

<zarr.hierarchy.Group '/'>

In [20]:
!du -sch /data/keeling/a/alfonso8/gpm/camp2ex/zarr_1/lores_rechunked.zarr

2.0M	/data/keeling/a/alfonso8/gpm/camp2ex/zarr_1/lores_rechunked.zarr
2.0M	total


In [21]:
!ls /data/keeling/a/alfonso8/gpm/camp2ex/zarr_1/lores_rechunked.zarr

DR	     ibeam_hires  ldrhh35	     scantime	    v_surfdc8
Xat_km	     ipc14	  ldrhh35SP	     sequence	    vel14
alt3D	     ipc35	  lon		     sfc_mask	    vel14SP
alt3d	     isc14	  lon3D		     sig14	    vel35
alt_nav      isc35	  lon3d		     sig14SP	    vel35SP
altsurf14    isurf	  look_vector	     sig35	    vel95s
altsurf35    isurf14	  look_vector_nadir  sig35SP	    z95s
azimuth      isurf35	  look_vector_radar  sig95s	    zhh14
beamnum      lat	  pitch		     surface_index  zhh14SP
cross_track  lat3D	  roll_		     time	    zhh35
drift	     lat3d	  s095s		     v_surf	    zhh35SP
elevation    ldrhh14	  s0hh14	     v_surf14
gsp_mps      ldrhh14SP	  s0hh35	     v_surf35


In [ ]:
!ls /data/keeling/a/alfonso8/gpm/camp2ex/zarr_1/lores_rechunked.zarr/zhh14

In [ ]:
ds_rechunked = xr.open_zarr('/data/keeling/a/alfonso8/gpm/camp2ex/zarr_1/lores_rechunked.zarr')

In [ ]:
ds_rechunked

In [ ]:
client.close()
cluster.close()